In [1]:
import numpy as np
import pandas as pd

## 1. Write a Python code to calculate the linear discriminant function for several classes. Your code should be able to predict the Y class based on the input 𝑥! values. (Page 31 in the lecture note)

In [2]:
def LDA_pred(dfX, dfy, prior, new_x):
    
    n_group = dfy.nunique()
    cov_list = []
    mean_list = []
    ldf_list = []
    n_list = []

    for i in range(n_group):
        X = dfX.iloc[dfy[dfy == i+1].index, :]
        n_list.append(len(X))
        mean_list.append(X.mean())
        cov_list.append(X.cov())

    def pooled_cov(n_list, cov_list, n_group):
        result = 0
        for x, y in zip(n_list, cov_list):
            product = (x-1)*y
            result += product

        pooled_cov = (1 / (np.sum(n_list)- n_group)) * result
        return pooled_cov

    s_p = pooled_cov(n_list, cov_list, n_group)

    coeff=[]
    intercept=[]
    d=[]
    for j in range(n_group):
        coeff.append(mean_list[j] @ np.linalg.inv(s_p))
        intercept.append(-1/2 * (mean_list[j] @ np.linalg.inv(s_p) @ mean_list[j].T))
        ldf = (mean_list[j] @ np.linalg.inv(s_p) @ np.array(new_x))- 1/2 * (mean_list[j] @ np.linalg.inv(s_p) @ mean_list[j].T) + np.log(prior[j])
        d.append(ldf)
    return_class = d.index(max(d))+1
    
    return intercept, coeff, d, max(d), return_class

## 2. Write a Python code to calculate the quadratic discriminant function for several classes. Your code should be able to predict the Y class based on the input 𝑥! values. (Page 35 in the lecture note)

In [3]:
def QDA_pred(dfX, dfy, prior, new_x):
    
    n_group = dfy.nunique()
    cov_list = []
    mean_list = []
    ldf_list = []
    n_list = []

    for i in range(n_group):
        X = dfX.iloc[dfy[dfy == i+1].index, :]
        n_list.append(len(X))
        mean_list.append(X.mean())
        cov_list.append(X.cov())

    d=[]
    for j in range(n_group):
        ldf = -1/2*np.log(np.linalg.det(cov_list[j])) - 1/2*(new_x - mean_list[j])@np.linalg.inv(cov_list[j])@(new_x - mean_list[j]).T + np.log(prior[j])
        d.append(ldf)
    return_class = d.index(max(d))+1
    
    return d, max(d), return_class

## 3. Write a Python code to perform the 'leave-one-out' method to calculate the accuracy of the LDA & QDA model you wrote in #1 & #2.

In [4]:
def LOO(dfX, dfy, prior, model):
    pred_y = []
    if model == 'LDA':
        for i in range(len(dfX)):
            newdata = dfX.iloc[i,:].tolist()
            X_train = dfX.drop(dfX.index[i]).reset_index().drop(['index'], axis = 1)
            y_train = dfy.drop(dfy.index[i]).reset_index().drop(['index'], axis = 1)[4]
            intercept, coeff, d_list, max_d, return_class = LDA_pred(X_train, y_train, prior, newdata)
            pred_y.append(return_class)
    else:
        for i in range(len(dfX)):
            newdata = dfX.iloc[i,:].tolist()
            X_train = dfX.drop(dfX.index[i]).reset_index().drop(['index'], axis = 1)
            y_train = dfy.drop(dfy.index[i]).reset_index().drop(['index'], axis = 1)[4]
            d_list, max_d, return_class = QDA_pred(X_train, y_train, prior, newdata)
            pred_y.append(return_class)
    result = pd.DataFrame({'original_y' : dfy, 'pred_y' : pred_y})
    F_result = result.loc[result['original_y']!=result['pred_y']]
    A_result = result.loc[result['original_y']==result['pred_y']]
    Accuracy_Rate = len(A_result) / len(result)
    error_rate = len(F_result) / len(result)
    return Accuracy_Rate, error_rate

## 4. Using Fisher's Iris data (Table 11.5), answer the following questions.

In [5]:
iris = pd.read_csv('iris.dat', header = None, delim_whitespace=True)
iris.head()

,0,1,2,3,4
0,5.1,3.5,1.4,0.2,1
1,4.9,3.0,1.4,0.2,1
2,4.7,3.2,1.3,0.2,1
3,4.6,3.1,1.5,0.2,1
4,5.0,3.6,1.4,0.2,1


### a. Is the assumption of a common covariance matrix reasonable in this case? (Use Python's 'statsmodels.stats.multivariate' module for this question).

In [6]:
from statsmodels.stats import multivariate as mv
group1 = iris.loc[iris[4]==1, 0:3]
group2 = iris.loc[iris[4]==2, 0:3]
group3 = iris.loc[iris[4]==3, 0:3]

cov1 = group1.cov() ; cov2 = group2.cov() ; cov3 = group3.cov()

test = mv.test_cov_oneway([cov1,cov2,cov3],[len(group1),len(group2),len(group3)])
print("Chi-Square Test statistic:",test.statistic_chi2, " , Pr > ChiSq:",test.pvalue_chi2)

# Since p-value is low, Do reject H0 => QDA

Chi-Square Test statistic: 140.94304992349774  , Pr > ChiSq: 3.352034178317213e-20


### b. Assuming that the populations are multivariate normal, calculate the quadratic discriminant scores with equal prior and equal misclassification cost. Classify the new observation 𝑥! = [5.0, 3.5, 1.75, 0.21]′ into one of populations

In [7]:
prior = [1/3, 1/3, 1/3]
new_x = [5.0, 3.5, 1.75, 0.21]
dfX = iris.iloc[:, 0:4]
dfy = iris[4]

d_list, max_d, pred_class = QDA_pred(dfX, dfy, prior, new_x)

print("Quadratic discriminant score : {}".format(d_list))
print("Predicted class for new observation : {}".format(pred_class))

Quadratic discriminant score : [3.4909763650152277, -47.30137268292963, -74.70869273851915]
Predicted class for new observation : 1


### c. Assuming equal covariance matrices and multivariate normal populations, calculate the linear discriminant function using your code in #1 above and compare its coefficients with those of Python's 'sklearn.discriminant_analysis' module.

In [8]:
prior = [1/3, 1/3, 1/3]
new_x = [5.0, 3.5, 1.75, 0.21]
dfX = iris.iloc[:, 0:4]
dfy = iris[4]

intercept, coeff, d_list, max_d, pred_class = LDA_pred(dfX, dfy, prior, new_x)
print("Intercept : {}".format(intercept))
print("Coefficeint : \n{}\n{}\n{}".format(coeff[0], coeff[1], coeff[2]))
print("LDF score : {}".format(d_list))
print("Predicted class for new observation : {}".format(pred_class))

Intercept : [-85.20985768500591, -71.75399511197409, -103.26970769778173]
Coefficeint : 
[ 23.54416672  23.5878705  -16.43063902 -17.39841078]
[15.69820908  7.07250984  5.21145093  6.4342292 ]
[12.44584899  3.68527961 12.76654497 21.07911301]
LDF score : [81.56262582121121, 40.86344967695457, -2.472528938799388]
Predicted class for new observation : 1


In [9]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
X = iris.iloc[:,0:4]
Y = iris.iloc[:,4]
lda = LDA(priors = prior, solver = 'eigen')
lda.fit(X,Y)

LinearDiscriminantAnalysis(priors=[0.3333333333333333, 0.3333333333333333,
                                   0.3333333333333333],
                           solver='eigen')

In [10]:
print("Intercept : {}".format(lda.intercept_))
print("Coefficeint : \n{}\n{}\n{}".format(lda.coef_[0], lda.coef_[1], lda.coef_[2]))

Intercept : [ -88.04744666  -74.31697465 -106.47586504]
Coefficeint : 
[ 24.02465992  24.06925561 -16.76595819 -17.75348039]
[16.01858069  7.21684677  5.31780708  6.56554   ]
[12.69984591  3.7604894  13.02708671 21.50929899]


### d. Calculate and compare the APER and the leave-one-out error rates for linear discriminant analysis (LDA) and quadratic discriminant analysis (QDA) using your code in #1,2,3. (Assume equal prior and equal misclassification cost).

In [11]:
def APER(dfX, dfy, prior, model):
    pred_y = []
    if model == 'LDA':
        for i in range(len(dfX)):
            newdata = dfX.iloc[i,:].tolist()
            intercept, coeff, d_list, max_d, return_class = LDA_pred(dfX, dfy, prior, newdata)
            pred_y.append(return_class)
    else:
        for i in range(len(dfX)):
            newdata = dfX.iloc[i,:].tolist()
            intercept, coeff, d_list, max_d, return_class = LDA_pred(dfX, dfy, prior, newdata)
            pred_y.append(return_class)
    result = pd.DataFrame({'original_y' : dfy, 'pred_y' : pred_y})
    F_result = result.loc[result['original_y']!=result['pred_y']]
    A_result = result.loc[result['original_y']==result['pred_y']]
    error_rate = len(F_result) / len(result)
    return error_rate

In [12]:
print("APER for LDA : {}".format(APER(dfX, dfy, prior, 'LDA')))
print("Leave-one-out error rate for LDA : {}".format(LOO(dfX, dfy, prior, 'LDA')[1]))

APER for LDA : 0.02
Leave-one-out error rate for LDA : 0.02


In [13]:
print("APER for QDA : {}".format(APER(dfX, dfy, prior, 'QDA')))
print("Leave-one-out error rate for QDA : {}".format(round(LOO(dfX, dfy, prior, 'QDA')[1],4)))

APER for QDA : 0.02
Leave-one-out error rate for QDA : 0.0267
